In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# tutti i link di questa sottosezione siano rilevanti all'analisi testuale
url = 'https://www.epicentro.iss.it/coronavirus/' 
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [3]:
#primo scraping di link della homepage del sito, che vanno aggiunti a una prima frontiera url
url_frontier = list()
url_frontier_completa = list()
for link in soup.find_all('a'):
    a = link.get('href')
    if "index" not in a and "/en" not in a and (url+a) not in url_frontier and not a.endswith(".pdf") and "http" not in a \
        and "aggiornamenti" not in a and "chi-siamo" not in a and "archivio" not in a and "newsletter" not in a:
        if "/coronavirus/" in a:
            url_frontier.append("https://www.epicentro.iss.it/" + a)
        elif a[0] == "/":
            url_frontier.append(url+a[1:])

In [4]:
#popolamento della frontiera a partire dai link trovati nella cella precedente (con dei filtri appropriati)
da_scartare = ["index", "/en", "#", "@", "?", "http", ":", "aggiornamenti", "chi-siamo", "archivio", "newsletter"]

for link in url_frontier:
  soup = BeautifulSoup(requests.get(link).text, 'html.parser')
  for linkage in soup.find_all('a'):
    a = linkage.get('href')
    if a == None or 'privacy' in a:
      continue
    link_valido = True
    for controllo in da_scartare:
        if controllo in a:
            link_valido = False
    if link_valido and not a.endswith(".pdf") and not a.endswith(".zip") and not a.endswith(".xslx"):
        if '../coronavirus' in a:
            url_completo = "https://www.epicentro.iss.it/" + a
            if url_completo not in url_frontier_completa:
                url_frontier_completa.append(url_completo)
                print("1 - Aggiunto " + url_completo)
            time.sleep(1)
        elif '/coronavirus' in a:
            url_completo = "https://www.epicentro.iss.it" + a
            if url_completo not in url_frontier_completa:
                url_frontier_completa.append(url_completo)
                print("2 - Aggiunto " + url_completo)
            time.sleep(1)
        else:
            url_completo = url + a
            if url_completo not in url_frontier_completa:
                url_frontier_completa.append(url_completo)
                print("3 - Aggiunto " + url_completo)
            time.sleep(1)

3 - Aggiunto https://www.epicentro.iss.it/coronavirus//
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/default
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sars-cov-2-sorveglianza-dati
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sars-cov-2-decessi-italia
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/../vaccini/covid-19
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sars-cov-2-survey-rsa
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sars-cov-2-flussi-dati-confronto-passi-pda
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/cosa-sono
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sintomi-diagnosi
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/trasmissione-prevenzione-trattamento
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/../../vaccini/covid-19
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sars-cov-2-gravidanza-parto-allattamento
3 - Aggiunto https://www.epicentro.iss.it/coronavirus/sars-cov-2-ipc
3 - A

In [5]:
len(url_frontier_completa)

68

In [6]:
for link in url_frontier_completa:
    url_frontier.append(link)
url_frontier = set(url_frontier)
len(url_frontier)

71

In [7]:
# la funzione per scaricare il testo e salvarlo in una struttura dati dedicata a partire da un url
def scaricaTesto(link, struttura_dati): 
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    titolo = soup.find('title')
    if titolo != None:
        titolo = titolo.get_text()
    else:
        titolo = "pagina non valida"
    testo = soup.find_all('p')
    testo_pulito = ""
    for p in testo:
        temp = p.get_text()
        temp.strip()
        testo_pulito += temp
    testo_pulito = testo_pulito.replace('\n','')
    testo_pulito = testo_pulito.replace('\t','')
    testo_pulito = testo_pulito.replace('\r','')
    testo_pulito = testo_pulito.replace('\xa0',' ')
    if len(testo_pulito) < 100: # serve per eliminare gli Error 404 per link mancanti o errati
        return
    if testo_pulito != "" or "Error" not in testo or titolo not in ["404", "Forbidden"]:
        struttura_dati.append(testo_pulito)
        link_url.append(link)
        time.sleep(5)

In [8]:
struttura_dati = list() # qui salviamo tutto il testo
link_url = list()
i = 1

for url in url_frontier:
    scaricaTesto(url, struttura_dati)
    print(str(i) + "/" + str(len(url_frontier)) + " - Scaricata la pagina " + url)
    i += 1

1/71 - Scaricata la pagina https://www.epicentro.iss.it/../coronavirus/sars-cov-2-servizi-vaccinali
2/71 - Scaricata la pagina https://www.epicentro.iss.it/coronavirus/sars-cov-2-animali-domestici
3/71 - Scaricata la pagina https://www.epicentro.iss.it/../coronavirus/sars-cov-2-analisi-studi-interventistici
4/71 - Scaricata la pagina https://www.epicentro.iss.it/../coronavirus/sars-cov-2-indagine-aie-contact-tracing
5/71 - Scaricata la pagina https://www.epicentro.iss.it/coronavirus/sars-cov-2-flussi-dati-confronto-passi-pda
6/71 - Scaricata la pagina https://www.epicentro.iss.it/coronavirus/sars-cov-2-gestione-stress
7/71 - Scaricata la pagina https://www.epicentro.iss.it/../coronavirus/trasmissione-prevenzione-trattamento
8/71 - Scaricata la pagina https://www.epicentro.iss.it/coronavirus//vaccini/VSN
9/71 - Scaricata la pagina https://www.epicentro.iss.it/../coronavirus/sars-cov-2-inquinamento-atmosferico
10/71 - Scaricata la pagina https://www.epicentro.iss.it/../coronavirus/sars-c

In [9]:
struttura_dati

["A causa della pandemia di COVID-19 (causata dal nuovo virus SARS-CoV-2) nei Paesi della Regione europea dell’Organizzazione Mondiale della Sanità (OMS) esiste il rischio che si possa verificare una riduzione generale delle normali attività vaccinali, sia per lo spostamento di risorse sanitarie verso le attività di controllo della pandemia, sia per le misure di distanziamento sociale imposte alla popolazione, che potrebbero portare alcune persone a decidere di rimandare le vaccinazioni programmate per se stessi o per i propri figli.  Le vaccinazioni sono una componente fondamentale dei servizi sanitari e una loro interruzione, anche se per un breve periodo, porterebbe a un accumulo di persone suscettibili e a un maggiore rischio di epidemie di malattie prevenibili da vaccino (VPD, vaccine preventable diseases) che possono causare decessi e portare una aumentata richiesta di risorse sanitarie. È fondamentale pertanto ridurre al minimo questo rischio, soprattutto in un sistema già prova

In [10]:
with open("scraping.txt", 'w', encoding='utf-8') as f:
    for articolo in struttura_dati:
        articolo_completo = ''
        for frase in articolo:
            articolo_completo += frase + ' '
        articolo_completo += '\n'    
        f.write(articolo_completo)